In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import keras
from keras.models import load_model
from PIL import Image
import numpy as np
from keras import backend as K
import tensorflow as tf

import glob

In [0]:
def dice_coef(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)
def bce_dice_loss(y_true, y_pred):
    return 0.5 * keras.losses.binary_crossentropy(y_true, y_pred) - dice_coef(y_true, y_pred)


In [0]:
width = 704
height = 1152
channel = 1

load_model = keras.models.load_model('/content/drive/My Drive/NTHU/HW03/weights-1206E10.hdf5',custom_objects={'bce_dice_loss': bce_dice_loss,'mean_iou':mean_iou})

filelist = glob.glob('/content/drive/My Drive/NTHU/HW03/ccpd6000/train_images/*.jpg')
for i in range(0, len(filelist),1 ):

    test_images = np.array([np.array(Image.open(filelist[i]).convert('L'))])
    test_images = test_images / 255.0
    test_images = test_images[:, 0:height, 0:width]
    test_images = test_images.reshape((test_images.shape[0], height, width,channel))

    result = load_model.predict(test_images)

    result = result * 255
    result[0] = result[0].astype(np.int_)


    pic = result[0].reshape(height,width)
    img = Image.fromarray(pic)
    img = img.convert('RGB')
    img.save("/content/drive/My Drive/NTHU/HW03/ccpd6000/ROI_img/"+str(i).zfill(4)  +".jpg")


filelist = glob.glob('/content/drive/My Drive/NTHU/HW03/ccpd6000/test_images/*.jpg')
for i in range(0, len(filelist),1 ):

    test_images = np.array([np.array(Image.open(filelist[i]).convert('L'))])
    test_images = test_images / 255.0
    test_images = test_images[:, 0:height, 0:width]
    test_images = test_images.reshape((test_images.shape[0], height, width,channel))

    result = load_model.predict(test_images)

    result = result * 255
    result[0] = result[0].astype(np.int_)


    pic = result[0].reshape(height,width)
    img = Image.fromarray(pic)
    img = img.convert('RGB')
    img.save("/content/drive/My Drive/NTHU/HW03/ccpd6000/test_ROI/"+str(i).zfill(4)  +".jpg")